## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mildura,AU,-34.2000,142.1500,52.07,80,100,9.73,light rain
1,1,Mataura,NZ,-46.1927,168.8643,33.48,96,15,2.91,few clouds
2,2,Mayumba,GA,-3.4320,10.6554,68.63,89,74,7.18,broken clouds
3,3,Puerto Madryn,AR,-42.7692,-65.0385,42.69,36,100,1.01,overcast clouds
4,4,Shymkent,KZ,42.3000,69.6000,91.99,14,0,17.90,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = int(input("What is your maximum temperature? "))
min_temp = int(input("what is your minimum temperature? "))

What is your maximum temperature? 90
what is your minimum temperature? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]



In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

Unnamed: 0             194
City                   194
Country                194
Lat                    194
Lng                    194
Max Temp               194
Humidity               194
Cloudiness             194
Wind Speed             194
Current Description    194
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,
14,Dingle,PH,76.08,broken clouds,10.9995,122.6711,
15,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,
18,Lodja,CD,78.85,overcast clouds,-3.4833,23.4333,
23,Assiniboia,CA,80.42,scattered clouds,49.6334,-105.9845,
29,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
32,San Andres,CO,80.62,light intensity drizzle,12.5847,-81.7006,
35,Dabeiba,CO,78.30,heavy intensity rain,7.0017,-76.2672,
37,San Cristobal,VE,80.78,broken clouds,7.7669,-72.2250,
38,Jamestown,US,86.05,few clouds,42.0970,-79.2353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

        lat = row["Lat"]
        lng = row["Lng"]
        
        params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        try:
        
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
            
        except IndexError:
            
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Dingle,PH,76.08,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
15,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Assiniboia,CA,80.42,scattered clouds,49.6334,-105.9845,Canalta Hotel Assiniboia
29,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,Paradise Inn
32,San Andres,CO,80.62,light intensity drizzle,12.5847,-81.7006,Hotel Arena Blanca
35,Dabeiba,CO,78.30,heavy intensity rain,7.0017,-76.2672,Hotel Cactus
37,San Cristobal,VE,80.78,broken clouds,7.7669,-72.2250,Pirineos
38,Jamestown,US,86.05,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
45,La Asuncion,VE,85.73,clear sky,11.0333,-63.8628,Colinas de Matasiete


In [15]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))